In [142]:
import numpy as np
import torch
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping 
import cmath
from torch import nn
import random
from torch.utils import data
import copy

In [2]:
class generator_t:
    def __init__(self,df,mu = 0, sigma = 1,samplesize = 1000, h = 0.01):
        self.df = df
        self.mu = mu
        self.sigma = sigma
        self.samplesize = samplesize
        self.frequency = h
    @staticmethod
    def cumulate(delta):
        data = torch.zeros(np.shape(delta))
        n = np.shape(delta)[0]
        for i in range(n-1):
            data[i+1] = data[i]+delta[i]
        return np.array(data)
    
    def generate_standard_t(self):
        d0 = np.random.standard_t(self.df, (self.samplesize,1))
        d1 = self.cumulate(d0)
        delta = torch.tensor(d0)
        data = torch.tensor(d1)
        delta.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
        return delta,data

### Characteristic Function

In [3]:
class Characteristic_function:
    def __init__(self,data):
        self.data = data   
    def ecf(self,t):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            return c,s                
        else:    
            m = len(t)
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            R = torch.zeros((m,2))
            for j in range(m):
                for i in range(n):
                    C[i] = math.cos(t[j]*self.data[i])
                    S[i] = math.sin(t[j]*self.data[i])
                s = float(torch.mean(S))
                c = float(torch.mean(C))
                R[j,0] = c
                R[j,1] = s
        return R
    def ecf_h(self,t,h):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            f = complex(c,s)
            q,w = cmath.polar(f)
            q = q**h
            w = w*h
            c = math.cos(w)*q
            s = math.sin(w)*q
            return c,s                
        else:    
            m = len(t)
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            R = torch.zeros((m,2))
            for j in range(m):
                for i in range(n):
                    C[i] = math.cos(t[j]*self.data[i])
                    S[i] = math.sin(t[j]*self.data[i])
                s = torch.mean(S)
                c = torch.mean(C)
                f = complex(c,s)
                q,w = cmath.polar(f)
                q = q**h
                w = w*h
                c = math.cos(w)*q
                s = math.sin(w)*q
                R[j,0] = c
                R[j,1] = s
        return R


In [4]:
class Characteristic_function:
    def __init__(self,data):
        self.data = data   
    def ecf(self,t):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            return c,s                
        else:    
            m = len(t)
            R = []
            for j in range(m):
                S = []
                C = []
                for i in range(n):
                    C.append(math.cos(t[j]*self.data[i]))
                    S.append(math.sin(t[j]*self.data[i]))
                S = torch.tensor(S,dtype = torch.float64)
                C = torch.tensor(C,dtype = torch.float64)
                s = float(torch.mean(S))
                c = float(torch.mean(C))
                R.append(c)
                R.append(s)
            R = torch.tensor(R,dtype = torch.float64)
        return R
    def ecf_h(self,t,h):
        n = len(self.data)
        if type(t)== int:
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            for i in range(n):
                C[i] = math.cos(t*self.data[i])
                S[i] = math.sin(t*self.data[i])
            s = torch.mean(S)
            c = torch.mean(C)
            f = complex(c,s)
            q,w = cmath.polar(f)
            q = q**h
            w = w*h
            c = math.cos(w)*q
            s = math.sin(w)*q
            return c,s                
        else:    
            m = len(t)
            C = torch.zeros((n,1))
            S = torch.zeros((n,1))
            R = torch.zeros((m,2))
            for j in range(m):
                for i in range(n):
                    C[i] = math.cos(t[j]*self.data[i])
                    S[i] = math.sin(t[j]*self.data[i])
                s = torch.mean(S)
                c = torch.mean(C)
                f = complex(c,s)
                q,w = cmath.polar(f)
                q = q**h
                w = w*h
                c = math.cos(w)*q
                s = math.sin(w)*q
                R[j,0] = c
                R[j,1] = s
        return R


In [146]:
num_feature = 20
range_feature = 10
num_t = 10
range_t = 5
num_labels = num_t
num_sample = 15
total = num_sample*num_feature
t = torch.linspace(-range_t,range_t,num_t,dtype = torch.float64)
#t = t.reshape((-1,len(t)))
x = torch.linspace(-range_feature,range_feature,num_feature,dtype = torch.float64)

In [147]:
#实例化(V =3)
Gen3 = generator_t(3,samplesize =total)

delta,path = Gen3.generate_standard_t()
Chf = Characteristic_function(delta)

In [148]:
a = []
for i in range(num_sample):
    delta,path = Gen3.generate_standard_t()
    Chf = Characteristic_function(delta)
    labels = Chf.ecf(t).reshape((1,-1))
    a.append([x.reshape((-1,len(x))),labels])


In [149]:
a

[[tensor([[-10.0000,  -8.9474,  -7.8947,  -6.8421,  -5.7895,  -4.7368,  -3.6842,
            -2.6316,  -1.5789,  -0.5263,   0.5263,   1.5789,   2.6316,   3.6842,
             4.7368,   5.7895,   6.8421,   7.8947,   8.9474,  10.0000]],
         dtype=torch.float64),
  tensor([[-0.0080,  0.0551,  0.0560,  0.0264,  0.1401,  0.0227,  0.2296,  0.0128,
            0.7348,  0.0196,  0.7348, -0.0196,  0.2296, -0.0128,  0.1401, -0.0227,
            0.0560, -0.0264, -0.0080, -0.0551]], dtype=torch.float64)],
 [tensor([[-10.0000,  -8.9474,  -7.8947,  -6.8421,  -5.7895,  -4.7368,  -3.6842,
            -2.6316,  -1.5789,  -0.5263,   0.5263,   1.5789,   2.6316,   3.6842,
             4.7368,   5.7895,   6.8421,   7.8947,   8.9474,  10.0000]],
         dtype=torch.float64),
  tensor([[ 0.0277, -0.0084,  0.0551,  0.0081,  0.0604,  0.0262,  0.2261, -0.0050,
            0.7364, -0.0345,  0.7364,  0.0345,  0.2261,  0.0050,  0.0604, -0.0262,
            0.0551, -0.0081,  0.0277,  0.0084]], dtype=torch.flo

In [150]:
df = pd.DataFrame(a)
features = df[0]
labels = df[1]
df[1][0][0]

tensor([-0.0080,  0.0551,  0.0560,  0.0264,  0.1401,  0.0227,  0.2296,  0.0128,
         0.7348,  0.0196,  0.7348, -0.0196,  0.2296, -0.0128,  0.1401, -0.0227,
         0.0560, -0.0264, -0.0080, -0.0551], dtype=torch.float64)

In [151]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [152]:
D_loader = data.DataLoader(a, 2, shuffle=False)

In [153]:
next(iter(D_loader)) 

[tensor([[[-10.0000,  -8.9474,  -7.8947,  -6.8421,  -5.7895,  -4.7368,  -3.6842,
            -2.6316,  -1.5789,  -0.5263,   0.5263,   1.5789,   2.6316,   3.6842,
             4.7368,   5.7895,   6.8421,   7.8947,   8.9474,  10.0000]],
 
         [[-10.0000,  -8.9474,  -7.8947,  -6.8421,  -5.7895,  -4.7368,  -3.6842,
            -2.6316,  -1.5789,  -0.5263,   0.5263,   1.5789,   2.6316,   3.6842,
             4.7368,   5.7895,   6.8421,   7.8947,   8.9474,  10.0000]]],
        dtype=torch.float64),
 tensor([[[-0.0080,  0.0551,  0.0560,  0.0264,  0.1401,  0.0227,  0.2296,
            0.0128,  0.7348,  0.0196,  0.7348, -0.0196,  0.2296, -0.0128,
            0.1401, -0.0227,  0.0560, -0.0264, -0.0080, -0.0551]],
 
         [[ 0.0277, -0.0084,  0.0551,  0.0081,  0.0604,  0.0262,  0.2261,
           -0.0050,  0.7364, -0.0345,  0.7364,  0.0345,  0.2261,  0.0050,
            0.0604, -0.0262,  0.0551, -0.0081,  0.0277,  0.0084]]],
        dtype=torch.float64)]

In [154]:
# 初始化参数定义


hidden_2, hidden_3 = 256, 256

W1 = nn.Parameter(torch.randn(
    num_feature, hidden_2, requires_grad=True,dtype = torch.float64) * 0.01)
b1 = nn.Parameter(torch.zeros(hidden_2, requires_grad=True,dtype = torch.float64))

W2 = nn.Parameter(torch.randn(
    hidden_2, hidden_3, requires_grad=True,dtype = torch.float64) * 0.01)
b2 = nn.Parameter(torch.zeros(hidden_3, requires_grad=True,dtype = torch.float64))

W3 = nn.Parameter(torch.randn(
    hidden_3, num_feature, requires_grad=True,dtype = torch.float64) * 0.01)
b3 = nn.Parameter(torch.zeros(num_feature, requires_grad=True,dtype = torch.float64))


params = [W1, b1, W2, b2, W3, b3]

In [155]:
#激活函数
def relu(X):
    a = torch.zeros_like(X)
    b = torch.max(X, a)
    return torch.tensor(b,dtype = torch.float64)

## $\phi(t) = \exp\{\frac{1}{20}\sum_{j=1}^{40}(e^{itx_j}-1-itx\mathbb{1}_{|x_j|\leq 1})\mathbf{\nu}_\theta(x_j)\}$
## $\phi(t) = \exp\{\frac{1}{20}\mathbf{Y(X,T)}^T\mathbf{\nu}_\theta(X)\}$

In [156]:
def net_1(X):
    X_clone = X.clone()
    X = X.reshape((-1, num_feature))
    H1 = relu(X@W1 + b1)  # 这里“@”代表矩阵乘法
    H2 = relu(H1@W2 + b2)
    V = (H2@W3 + b3)
    return V


In [157]:
def expcostx(x):
    a = t.reshape((-1,1))@x.reshape((1,-1))
    b = (torch.cos(a)-1)@net_1(x).reshape((-1,1))
    c = torch.exp((range_feature*2/num_feature)*b)
    return c
def I(x):
    y = copy.deepcopy(x)
    y[abs(x)>1]=0
    return y
    
def realtx(x):
    a1 = torch.sin(t.reshape((-1,1))@x.reshape((1,-1))) 
    a2 = t.reshape((-1,1))@I(x).reshape((1,-1))
    a3 = a1+a2
    a4 = a3@net_1(x).reshape((-1,1))
    a5 = (range_feature*2/num_feature)*a4
    a6 = torch.cos(a5)
    return a6
def imagetx(x):
    a1 = torch.sin(t.reshape((-1,1))@x.reshape((1,-1))) 
    a2 = t.reshape((-1,1))@I(x).reshape((1,-1))
    a3 = a1+a2
    a4 = a3@net_1(x).reshape((-1,1))
    a5 = (range_feature*2/num_feature)*a4
    a6 = torch.sin(a5)
    return a6

In [161]:
expcostx(b)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10384\2169779723.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(b,dtype = torch.float64)


tensor([[1.0367],
        [1.0040],
        [0.9985],
        [1.0095],
        [1.0200],
        [1.0200],
        [1.0095],
        [0.9985],
        [1.0040],
        [1.0367]], dtype=torch.float64, grad_fn=<ExpBackward0>)

In [162]:
expcostx(b)*realtx(b)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10384\2169779723.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(b,dtype = torch.float64)


tensor([[1.0364],
        [1.0040],
        [0.9985],
        [1.0093],
        [1.0200],
        [1.0200],
        [1.0093],
        [0.9985],
        [1.0040],
        [1.0364]], dtype=torch.float64, grad_fn=<MulBackward0>)

In [159]:
imagetx(b)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10384\2169779723.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(b,dtype = torch.float64)


tensor([[-0.0212],
        [-0.0047],
        [-0.0068],
        [-0.0152],
        [ 0.0039],
        [-0.0039],
        [ 0.0152],
        [ 0.0068],
        [ 0.0047],
        [ 0.0212]], dtype=torch.float64, grad_fn=<SinBackward0>)

In [127]:
b = torch.linspace(-10,10,20,dtype = torch.float64)
b

tensor([-10.0000,  -8.9474,  -7.8947,  -6.8421,  -5.7895,  -4.7368,  -3.6842,
         -2.6316,  -1.5789,  -0.5263,   0.5263,   1.5789,   2.6316,   3.6842,
          4.7368,   5.7895,   6.8421,   7.8947,   8.9474,  10.0000],
       dtype=torch.float64)

In [124]:
z = torch.tensor([1,2,3,-1,-2,-3])
z[abs(z)>1]=10
z

tensor([ 1, 10, 10, -1, 10, 10])

In [254]:
(costx(a[0][0])-1)@net_1(a[0][0]).reshape((-1,1))

/var/folders/ym/35040q6j5x912qskn704_hy80000gn/T/ipykernel_44486/2169779723.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(b,dtype = torch.float64)


tensor([[ 0.7638],
        [ 0.6014],
        [ 0.4390],
        [ 0.2766],
        [ 0.1143],
        [-0.0481],
        [-0.2105],
        [-0.3729],
        [-0.5353],
        [-0.6977]], dtype=torch.float64, grad_fn=<MmBackward0>)

In [177]:
# Levy-Khintchine representation
def I(x):
    I = torch.ones(x.shape,dtype=torch.float64)
    x = abs(x)
    for i in range(len(x)):
        if x[i]>1:
            I[i] = 0
    return I
# indicator function
def z(x):
    if abs(x)>1:
        return 0 
    else:
        return 1
def Euler_exp(x,t):
    n = len(x)
    m = len(t)
    y = torch.zeros((m,n),dtype = torch.float64)
    for i in range(m):
        for j in range(n):
            y[i][j]=torch.exp(x[j]*t[i]*1j)-1-1j*t[i]*x[j]*z(x[j])
            
#    s = torch.exp(1j*t.reshape((len(t),-1))@x.reshape((-1,len(x))))-1-1j*(t.reshape((len(t),-1))@x.reshape((-1,len(x))))@I(x).reshape((len(I(x)),-1))
    return y 

In [178]:
#网络定义

def net(X):
    X_clone = X.clone()
    X = X.reshape((-1, num_feature))
    H1 = relu(X@W1 + b1)  # 这里“@”代表矩阵乘法
    H2 = relu(H1@W2 + b2)
    V = (H2@W3 + b3)
#    return V
    Chf = Euler_exp(X_clone,t)@V.t()
    re = Chf.real
    im = Chf.imag
    r = torch.zeros(labels.shape)
    for i in range(len(Chf)):
        r[i][0] = re[i]
        r[i][1] = im[i]
    return r


In [174]:
net(x)

RuntimeError: expected scalar type Float but found ComplexFloat